# A first example of a numerical simulation

test

In this exercise, you will conduct a first instance of numerical simulation of the time evolution of a system. We will solve an advection equation of the form

$$\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0  \tag{1}$$

with $a$ being a __constant__. In subsequent exercises, more general cases will be considered, with $a$ no longer constant.

## 1 – The $a = const$ case: the mathematical problem

We would like to solve equation (1) numerically for $x  [x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, periodic boundary conditions and with the initial condition:

$$u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2) \tag{2}$$

We start by representing that function in the given interval.

In [ ]:
# importing libraries
import numpy as np 
import matplotlib.pyplot as plt 
from nm_lib import nm_lib as nm


In [ ]:
nint = 64
nump = nint +1 


x0 = -2.6
xf = 2.6 
x = np.linspace(x0, xf, nump)


def initial_u(xx):

    r"""
    Computing the initial condition of u_i
    
    Requires
    ----------
    Numpy

    Parameters
    ----------
    xx : `array`
        Spatial array

    Returns
    ------- 
    initial condition : `array`
        initial condition of u_i
    """

    return np.cos(6*np.pi*xx/5)**2/np.cosh(5*xx**2)


u0 = initial_u(x)

In [ ]:
plt.plot(x,u0)

plt.title("Initial condition")
plt.xlabel("x")
plt.ylabel("u_0")
plt.show()

## 2 – Spatial derivative.

We discretize the initial condition by subdividing the spatial domain into $nint=64$ equal intervals. The function will therefore be sampled with $nump=65$ points, namely $(u_0, u_1, u_2, ... , u_{nump−1})$ at equidistant values of the abscissa, $(x_0, x_1, x_2, ..., x_{nump−1})$. Let us calculate the spatial derivative of the function through non-centered finite differences of the form:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i}}{\Delta x}  \tag{3}$$

On the basis of the experience gained with the previous batch of exercises (ex. 1), which order of approximation can we expect now for the calculation of the derivative since we are using non-centered finite differences?

In [ ]:
dudx = nm.deriv_dnw(x, u0)

In [ ]:
plt.plot(x,dudx)

plt.title("Spatial derivative")
plt.xlabel("x")
plt.ylabel("du/dx")
plt.show()

Since we are using the upwind scheme for calculating the derivative, we expect the order of approximation to be 2 as it is similar to the downwind scheme we used earlier.

## 3 – Time advance.

We want to calculate an approximation to the value of the function at times later than $t = t_0$, for instance, at time $t_0 + \Delta t$. To that end, we carry out a discretization of the time axis, calculating approximate values for the time derivative as follows:

$$\left(\frac{\partial u}{\partial t}\right)_{\underset{t=t_0}{x=x_i}} \rightarrow \frac{u_i(t_0 + \Delta t)-u_i(t_0)}{\Delta t}\tag{4}$$

Using now the differential equation (1) and expression (3), we can calculate an approximate value for the function $u$ at $x_i$ and time $t_0 + \Delta t$ as follows:

$$u_i(t_0+\Delta t) = u_i(t_0) - a \frac{u_{i+1}(t_0) - u_i(t_0)}{\Delta x}\Delta t  \tag{5}$$

Using (5), calculate $u_i$ at time $t_0 + \Delta t$ at all points of the sample excluding the rightmost point (i.e., excluding $x_{nump−1}$). Use  $\Delta t = 0.98 \Delta x/|a|$ and a = −1. Fill in `nm_lib` the functions `step_adv_burgers` and `cfl_adv_burger`. 

In [ ]:
a = -1 
dt, rhs = nm.step_adv_burgers(x, u0, a, ddx=nm.deriv_dnw)
ui = u0 + rhs*dt

In [ ]:

plt.plot(x,u0, label=r"$u_0$")
plt.plot(x,ui, label=r"$ u_i$ at $t_0 + \Delta t$")
plt.xlabel("x")
plt.ylabel("u_i")
plt.legend()
plt.show()

In the plot above, we show the curve from the initial condition as well as the result of a single step forward in time. We see that the curve is shifted towards left. We did this using the upwind scheme. 

## 4 – The boundaries.

To calculate the function at the rightmost point of the interval, we use a periodicity condition on $u$: $u_{nump−1}(t_0 + \Delta t) = u_0(t_0 + \Delta t)$. Consider cutting the ill-calculated (or missing) grid points and using `NumPy.pad` to add different boundary conditions. Use `wrap`


We implemented this directly in `evolv_adv_burger`, using `numpy.pad` and `wrap`


## 5 – Subsequent steps in time.

Having calculated the values $u_i$, $i = 0, 1, 2, ...,nump−1,$ at time $t_0+\Delta t$, we can carry out another step in time of size $\Delta t$, following exactly the method just explained. In general, if we have calculated the value of the $u$ function at $x_i$ and time $n\Delta t$, which we will call $u^n_i$, we can carry out the next step in time, of size $\Delta t$, through the expression:

$$u_i^{n+1} = u_i^n - a \frac{u_{i+1}^n - u_i^n}{\Delta x}\Delta t  \tag{6}$$

Periodic boundary conditions must be applied, as explained in 4 above, to finish the calculation at each timestep. Fill in nm_lib the function `evolv_adv_burgers`. 

Carry out many steps in time so you can clearly understand the mathematical nature of the solution of the equation. For example, in Python, you can _matplotlib.animation_ to see the evolution. 

In [ ]:
nt = 300
t, un = nm.evolv_adv_burgers(x, u0, nt,a, ddx=nm.deriv_dnw, bnd_limits=[0,1])

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.ioff()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    axes.plot(x,un[:,0])

def animate(i):
 
    axes.clear()
    axes.plot(x,un[:,i])
    axes.set_title('t=%.2f'%t[i])
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    
anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
HTML(anim.to_jshtml())


In [ ]:
plt.close()

Using the upwind scheme, and the implemented `evolv_adv_burgers` function in `nm_lib.py`, we evolve our system further in time. Again we see the peaks moving towards left, as was first shown in task 3 with the single step. 
As the function evolves in time, we see the peaks becoming more and more diffuse. 

## 6 – Comparison with the exact solution.

Through one of the theory sessions, we know the exact solution of eq. (1) for the initial condition (2). Draw the exact solution on top of the numerical one using a dashed line. Then, explain the mathematical behavior of the solution. __Important note__: consider that the initial condition is eq (2) in $[x_0, x_f]$, _with periodic conditions at the boundaries_. In other words, the initial condition consists of an infinite repetition of the function (2) you represented between $[x_0, x_f]$ next to each other. Use this fact when comparing your numerical solution with the analytical one. __Hint__: if you consider points starting within the interval $(x_0, x_f)$ and moving with speed $a$, they will go outside the domain after some time. Use mod operator (\% in Python) to bring them back into the domain respecting the periodicity of the problem or _numpy.pad_ to pad ghost points, i.e., points out of the numerical domain, which will allow defining the boundaries, at both ends of the numerical domain. NumPy.pad allows various types of boundaries.

Add a CI/CD pipeline to run this test and validate each push commit. This lets us know if a specific submitted change damages the existing code. For this, fill in ./github/workflows/test.yml

In [ ]:
nt = 400
t, un = nm.evolv_adv_burgers(x, u0, nt,a, ddx=nm.deriv_dnw, bnd_limits=[0,1])


In [ ]:
def analytical(xx, nt, a, t): 

    r"""
    Computes the analytical solution of Eq.(1)
    
    Requires
    ----------
    Numpy

    Parameters
    ----------
    xx : `array`
        spatial array.
    nt : `array`
        number of time steps
    a : `float` or `array`
        Either constant, or array which multiply the right hand side of the Burger's eq.
    t : `array`
        time

    Returns
    ------- 
    A : `array`
        analytical solution with periodic boundaries
    """

    A = np.zeros((len(xx), nt))
    U = np.zeros((len(xx), nt))

    for i in range(nt):
        U[:,i] = (xx-a*t[i])

        """
        The -a*t/5.2)[-1], in the following range is to round up how many times the 
        function "passes the wall". This way we know how many times we need to
        wrap the analytical function
        """
        for j in reversed(range(int(np.round(-a*t/5.2)[-1]))): 
            U[np.where(U[:,i] > 2.6+j*5.2)[0], i]  = U[np.where(U[:,i] > 2.6 + 5.2*j)[0], i] - (j+1)*5.2

        A[:,i] = initial_u(U[:,i])

    return A

A = analytical(x, nt, a, t)

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.ioff()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    axes.plot(x,un[:,0])
    axes.plot(x,A[:,0])

def animate(i):
 
    axes.clear()
    axes.plot(x,un[:,i], label="Numerical")
    axes.plot(x,A[:,i], "--", label="Analytical")
    axes.set_title('t=%.2f'%t[i])
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    axes.legend()
    
anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
HTML(anim.to_jshtml())

In [ ]:
plt.close()

In the animation above, we show both the numerical and analytical solution as they evolve in time. While the analytical solution only shifts its position as the time goes, the numerical solution seems to flatten out and become more diffuse. This is noticeable from the amplitude of the peaks. 

Though I do not understand it completely, the numerical solution of the advective equation goes under "Hyper Diffusive" numerical schemes. This means the equation is difficult to solve numerically, as we can see from the deviation from the analytical solution, due to Numerical Diffusion. I think this could be due to different velocities within the medium, which makes the numerical solution more diffusive. 

## 7 – Resolution increase in space and time. 

Let us repeat the calculation of the previous paragraphs, increasing the number of space intervals by factors of 2. Check if the numerical solution gets increasingly close to the analytical solution. Important: the comparison must be made _for the same values of $x$ and $t$ for all resolutions you check_. Choose a fixed time when the traveling function has already gone through the whole domain a few times. 

The comparison is made for time = 12 in all the different cases. For the spatial values, we compare the numerical and analytical solution at the central peak.

In [ ]:
nint_64 = 64
nump_64 = nint_64 + 1
nt_64 = 400

x_64 = np.linspace(x0, xf, nump_64)
u0_64 = initial_u(x_64)
t_64, un_64 = nm.evolv_adv_burgers(x_64, u0_64, nt_64,a, ddx=nm.deriv_dnw, bnd_limits=[0,1])

analytical_64 = analytical(x_64, nt_64, a, t_64)

time = np.argmin(np.abs(t_64-12))
peak = np.argmax(un_64[:,time])

In [ ]:
nint_128 = 128
nump_128 = nint_128 + 1
nt_128 = 400

x_128 = np.linspace(x0, xf, nump_128)
u0_128 = initial_u(x_128)
t_128, un_128 = nm.evolv_adv_burgers(x_128, u0_128, nt_128,a, ddx=nm.deriv_dnw, bnd_limits=[0,1])

analytical_128 = analytical(x_128, nt_128, a, t_128)

time_128 = np.argmin(np.abs(t_128-12))
peak_128 = np.argmax(un_128[:,time_128])


In [ ]:
nint_256 = 256 
nump_256  = nint_256  + 1
nt_256 = 650

x_256  = np.linspace(x0, xf, nump_256 )
u0_256 = initial_u(x_256)
t_256 , un_256  = nm.evolv_adv_burgers(x_256 , u0_256 , nt_256 ,a, ddx=nm.deriv_dnw, bnd_limits=[0,1])

analytical_256  = analytical(x_256 , nt_256 , a, t_256 )

time_256  = np.argmin(np.abs(t_256 -12))
peak_256  = np.argmax(un_256 [:,time_256 ])

In [ ]:
nint_512 = 512
nump_512  = nint_512  + 1
nt_512 = 1250

x_512  = np.linspace(x0, xf, nump_512 )
u0_512 = initial_u(x_512)
t_512 , un_512  = nm.evolv_adv_burgers(x_512 , u0_512 , nt_512 ,a, ddx=nm.deriv_dnw, bnd_limits=[0,1])

analytical_512  = analytical(x_512 , nt_512 , a, t_512 )

time_512  = np.argmin(np.abs(t_512 -12))
peak_512  = np.argmax(un_512 [:,time_512 ])

In [ ]:
diff_64 = np.abs(un_64[peak, time] - analytical_64[peak, time])
diff_128 = np.abs(un_128[peak_128, time_128] - analytical_128[peak_128, time_128])
diff_256 = np.abs(un_256[peak_256, time_256] - analytical_256[peak_256, time_256])
diff_512 = np.abs(un_512[peak_512, time_512] - analytical_512[peak_512, time_512])

print("Absolute difference, nint = 64: ", diff_64)
print("Absolute difference, nint = 128: ", diff_128)
print("Absolute difference, nint = 256: ", diff_256)
print("Absolute difference, nint = 512: ", diff_512)

We see that when we increase the number of points (nump), the absolute difference between the numerical and analytical solution decreases as expected. We also see this in the plot below, where we compare the numerical solutions for the different `nump` values. Here we compare the numerical solutions to the analytical solution for `nint` = 512, which is the largest value we compare with. 
For the larger number of `nint/nump`, we see the numerical solution is closer to the analytical solution. 

In [ ]:
plt.plot(x_64, un_64[:, time], label="nint = 64")
plt.plot(x_128, un_128[:,time_128], label="nint = 128")
plt.plot(x_256, un_256[:,time_256], label="nint = 256")
plt.plot(x_512, un_512[:,time_512], label="nint = 512")
plt.plot(x_512, analytical_512[:,time_512], "--", label="analytical, nint = 512")
plt.title('t=%.2f'%t_512[time_512])
plt.legend()
plt.show()